In [ ]:
# -*- coding: utf-8 -*-
import os 
import numpy as np
import matplotlib.pyplot as plt
import glob
from tkinter import filedialog as fd
import time
import datetime
from scipy.signal import find_peaks
import math 
from sklearn.linear_model import LinearRegression

###############################################################################
########## Code Body ##########################################################
###############################################################################
    
    
workingPath = fd.askdirectory()
os.chdir(workingPath)

if os.path.isfile("SynthesisFile.txt"):
    os.remove("SynthesisFile.txt")
    
for fileName in sorted(glob.glob("*.IP")):
    st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')  
                 
    #Read the CSV that contains all the data
    currentFile = open(fileName,'r')
    fileLines = currentFile.readlines() 
    fileNameOut = fileName.rstrip(".txt")
    currentFile.close()
    print(st)
    print("Processing: " + fileNameOut)
        
    Field = np.array([])
    hallVolt = np.array([])
    gate_voltage = np.array([])
    theta_M = np.array([])
    sin_theta = np.array([])
    cos_theta = np.array([])
    channel_voltage = np.array([])
    voltage_Counter = 1
    fit = np.array([])
    v_hall = np.array([])

    for lineIndex in range(len(fileLines)):
        currentLine = fileLines[lineIndex]
        dataSegment = currentLine.split()
        Field = np.append(Field, float(dataSegment[9])) #This is milli Tesla
        hallVolt = np.append(hallVolt, float(dataSegment[5])) #This is milli Volt
        channel_voltage = np.append(channel_voltage, float(dataSegment[2])) #This is Volt
        VG = float(dataSegment[6]) - float(dataSegment[2])/2
        gate_voltage = np.append(gate_voltage, np.round(VG, 2)) #This is Volt
        if lineIndex > 1 and VG != gate_voltage[lineIndex-1]:
            voltage_Counter = voltage_Counter + 1
            
    HV = hallVolt-((max(hallVolt))+min(hallVolt))/2
    # Split up field array as well ()  ##this line is written by my mentor
    HVN = HV/((max(HV)-min(HV))/2)
    
    for index in range(len(HVN)):
        if HVN[index] < 0:
            theta_M = np.append(theta_M, math.acos(HVN[index])-np.pi)
        else:
            theta_M = np.append(theta_M, math.acos(HVN[index]))

    for angle in range(len(theta_M)):
        sin_theta = np.append(sin_theta, math.sin(theta_M[angle]))
        cos_theta = np.append(cos_theta, math.cos(theta_M[angle]))

    #From here on issues in code
    x = sin_theta ** 2
    RHS = abs(Field * ((math.cos(86*np.pi/180)/cos_theta) - (math.sin(86*np.pi/180)/sin_theta)))
    array = np.column_stack([x, RHS, gate_voltage])
    
    dup = np.unique(gate_voltage)
    for i in dup:
        xFit = np.array([])
        RHSFit = np.array([])
        x_new = []
        RHS_new = []
        for a,b,c in array:
            if c == i: # change specific gate_voltage value
                x_new.append(a)
                RHS_new.append(b)
        x_new = np.array(x_new)
        RHS_new = np.array(RHS_new)
        for index in range(len(x_new)):
            if x_new[index] > 0.2 and x_new[index] < 0.7:
                xFit = np.append(xFit, x_new[index])
                RHSFit = np.append(RHSFit, RHS_new[index])

        model = LinearRegression().fit(xFit.reshape(-1,1), RHSFit)
        regression_line = xFit*model.coef_ + model.intercept_

        m = (((np.mean(xFit) * np.mean(RHSFit)) - np.mean(xFit*RHSFit)) / (np.mean(xFit)**2 -np.mean(xFit*RHSFit)))
        b = np.mean(RHSFit) - m*np.mean(xFit)
        regression_line = xFit*m + b

        print(f'======================= {i} ============================')
        plt.figure(1)
        plt.scatter(xFit, RHSFit, c = 'green')
        plt.plot(xFit, regression_line, c = 'r')
        figureName = '04. RHS vs LHS.png'
        plt.title("||µ0_H|| (cos\u03B8_H/cos\u03B8_M - sin\u03B8_H/sin\u03B8_M vs. sin²\u03B8M")
        plt.xlabel("sin²\u03B8M [arb. unit]")
        plt.ylabel("||\u03BC H|| (cos\u03B8_H/cos\u03B8_M - sin\u03B8_H/sin\u03B8_M [mT]")
        plt.savefig(os.path.join(workingPath, figureName), dpi = 200, format = 'png')
        plt.show()
        plt.close(1)
        

In [23]:
# -*- coding: utf-8 -*-
import os 
import numpy as np
import matplotlib.pyplot as plt
import glob
from tkinter import filedialog as fd
import time
import datetime
from scipy.signal import find_peaks
import math 
from sklearn.linear_model import LinearRegression

###############################################################################
########## Code Body ##########################################################
###############################################################################
    
    
workingPath = fd.askdirectory()
os.chdir(workingPath)

if os.path.isfile("SynthesisFile.txt"):
    os.remove("SynthesisFile.txt")
    
for fileName in sorted(glob.glob("*.IP")):
    st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')  
                 
    #Read the CSV that contains all the data
    currentFile = open(fileName,'r')
    fileLines = currentFile.readlines() 
    fileNameOut = fileName.rstrip(".txt")
    currentFile.close()
    print(st)
    print("Processing: " + fileNameOut)
        
    Field = np.array([])
    hallVolt = np.array([])
    gate_voltage = np.array([])
    theta_M = np.array([])
    sin_theta = np.array([])
    cos_theta = np.array([])
    channel_voltage = np.array([])
    voltage_Counter = 1
    fit = np.array([])
    v_hall = np.array([])

    for lineIndex in range(len(fileLines)):
        currentLine = fileLines[lineIndex]
        dataSegment = currentLine.split()
        Field = np.append(Field, float(dataSegment[9])) #This is milli Tesla
        hallVolt = np.append(hallVolt, float(dataSegment[5])) #This is milli Volt
        channel_voltage = np.append(channel_voltage, float(dataSegment[2])) #This is Volt
        VG = float(dataSegment[6]) - float(dataSegment[2])/2
        gate_voltage = np.append(gate_voltage, np.round(VG, 2)) #This is Volt
        if lineIndex > 1 and gate_voltage[lineIndex] != gate_voltage[lineIndex-1]:
            voltage_Counter = voltage_Counter + 1
            
    HV = hallVolt-((max(hallVolt))+min(hallVolt))/2
    array = np.column_stack([Field, HV, gate_voltage])
    np.savetxt('test.txt', array, delimiter=' ')


2020-04-08 17:37:56
Processing: AHE_3V_IP_RawData.ip


In [91]:
len(index)


6

In [111]:
len(index)

6

In [110]:
import pandas as pd
import numpy as np
df = pd.read_csv('test.txt', delim_whitespace=True, header=None)
df.columns=['Field','hall_voltage','gate_voltage']
index = df.gate_voltage.diff()[df.gate_voltage.diff() != 0].index.values
a=df.loc[index[len(index)-6]:index[len(index)-5]-1,:].reset_index(drop=True)
b=df.loc[index[len(index)-5]:index[len(index)-4]-1,:].reset_index(drop=True)
c=df.loc[index[len(index)-4]:index[len(index)-3]-1,:].reset_index(drop=True)
d=df.loc[index[len(index)-3]:index[len(index)-2]-1,:].reset_index(drop=True)
e=df.loc[index[len(index)-2]:index[len(index)-1]-1,:].reset_index(drop=True)
f=df.loc[index[len(index)-1]:df.tail(1).index.item(),:].reset_index(drop=True)
new = pd.concat([a,b,c,d,e,f],axis=1)
np.savetxt('new.txt', new, delimiter=' ')

C:\Users\spark\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()


,Field,hall_voltage,gate_voltage,Field,hall_voltage,gate_voltage,Field,hall_voltage,gate_voltage,Field,hall_voltage,gate_voltage,Field,hall_voltage,gate_voltage,Field,hall_voltage,gate_voltage
0,1026.604533,10.815801,-0.0,1022.974014,10.624116,1.5,1022.586465,10.816352,-1.5,1022.620320,10.406505,3.0,1022.968769,11.054813,-3.0,1022.638083,10.814657,0.0
1,1023.559332,10.813545,0.0,1019.548774,10.557031,1.5,1019.711852,11.009153,-1.5,1019.660234,10.580701,3.0,1019.956112,10.736302,-3.0,1020.166397,10.887943,-0.0
2,1020.266414,10.912764,0.0,1016.636252,10.869907,1.5,1016.568780,10.805078,-1.5,1016.644359,10.409323,3.0,1016.767621,11.165308,-3.0,1016.733885,11.077359,0.0
3,1016.453385,11.003526,0.0,1012.992263,10.634262,1.5,1013.228297,10.749272,-1.5,1013.161540,10.542368,3.0,1012.922287,10.926281,-3.0,1012.878418,10.994490,-0.0
4,1012.972951,10.951097,0.0,1009.641051,10.911622,1.5,1009.794116,10.831015,-1.5,1009.893298,10.751516,3.0,1009.245515,10.868217,-3.0,1010.256171,11.024366,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,1004.120350,10.997314,0.0,1004.013181,10.781961,1.5,1004.410982,10.909364,-1.5,1002.636552,10.790981,3.0,1003.390074,11.094836,-3.0,1003.963351,10.522073,0.0
797,1009.015322,10.811843,0.0,1008.625388,10.938681,1.5,1008.871675,11.074540,-1.5,1007.559776,10.336606,3.0,1007.746100,10.948262,-3.0,1007.814765,11.086942,-0.0
798,1013.095379,10.794931,-0.0,1013.208866,10.859759,1.5,1013.581634,11.203071,-1.5,1011.880517,10.779142,3.0,1013.249516,10.834387,-3.0,1013.069391,10.873849,-0.0
799,1017.514706,10.741375,0.0,1018.101692,10.536171,1.5,1017.941952,10.971374,-1.5,1016.600251,10.798874,3.0,1017.529488,11.226752,-3.0,1017.739534,10.884561,0.0


In [71]:
df.tail(1).index.item()

C:\Users\spark\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `item` has been deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


4805

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('test.txt', delim_whitespace=True, header=None)
df.columns=['Field','hall_voltage','gate_voltage']
unique = np.unique(df.gate_voltage)
df_new = df.sort_values(by=['gate_voltage']).reset_index(drop=True)
df_new.set_index('gate_voltage')
grouped = df_new.groupby('gate_voltage')
gp=[]
for name,group in grouped:
    gp.append(group)
    print(name)

gp[0].reset_index(drop=True)
gp[1].reset_index(drop=True)
gp[2].reset_index(drop=True)
gp[3].reset_index(drop=True)
gp[4].reset_index(drop=True)

In [ ]:
a=gp[0].reset_index(drop=True)
b=gp[1].reset_index(drop=True)
c=gp[2].reset_index(drop=True)
d=gp[3].reset_index(drop=True)
e=gp[4].reset_index(drop=True)
new = pd.concat([a,b,c,d,e],axis=1)
new

In [ ]:
# df_new=[]
# for item in unique:
#     gv=df.loc[df['gate_voltage']==item].reset_index(drop=True)
#     np.savetxt('gv_sep'+str(i)+'.txt', gv, delimiter=' ')


In [ ]:
# a = len(df_new)
# n=[]
# for j in range(a):
#     new = pd.concat([df_new[j]],axis=1)
#     n.append(new)
# n

In [ ]:
# g_0=df.loc[df['gate_voltage']==0].reset_index(drop=True)
# g_m3=df.loc[df['gate_voltage']==-3].reset_index(drop=True)
# g_m1P5=df.loc[df['gate_voltage']==-1.5].reset_index(drop=True)
# g_3=df.loc[df['gate_voltage']==3].reset_index(drop=True)
# g_1P5=df.loc[df['gate_voltage']==1.5].reset_index(drop=True)

In [ ]:
# df_new = pd.concat([g_0,g_1P5,g_m1P5,g_3,g_m3],axis=1)

In [ ]:
# np.savetxt('gv_seperated.txt', df_new, delimiter=' ')